# `Извлечение именованных сущностей и фактов`

> При постановке задачи по извлечению информации было замечено, что необходимо различать такие информационные единицы, как персона, организация, локация и числовые выражения, включающие в себя время, даты, деньги и проценты.





# Named-entity recognition, NER

---






# Подзадачи

---



1.  Обнаружить, что какая-то последовательность слов — это именованная сущность
2.   Понять, к какому классу (имя человека, название организации, город и т.п.) эта именованная сущность относится.



# Применение

---



1.   структуризация неструктурированных данных
2.   перевод текстов
3.   создание краткого изложения
4.   извлечение субъективной информации



# Оценка NER
```
        количество верно выделенных сущностей 
  P=    ______________________________________
        количество всех выделенных сущностей 
```
```
        количество верно выделенных сущностей
  R=    ______________________________________
        количество сущностей в корпусе

```



```
      2PR
F2=  _____
      P+R
```





In [28]:
pip install pymorphy2
pip install stanza
pip install natasha
pip install pullenti_wrapper

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 9.8 MB/s 


In [ ]:
import nltk
import pymorphy2
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import stanza
from natasha import NamesExtractor
from pullenti_wrapper.processor import Processor, PERSON
from sklearn.metrics import f1_score
stanza.download('ru')
nltk.download('punkt')
stanza_nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
morph = pymorphy2.MorphAnalyzer()

2021-09-29 13:49:57 INFO: Downloading default packages for language: ru (Russian)...


In [ ]:
def name_recognize_nltk(text):
    prob_thresh = 0.4
    text = str(text)
    result = ''
    global morph
    for word in nltk.word_tokenize(text):
       for p in morph.parse(word):
            if ('Name' in p.tag and p.score >= prob_thresh) or ('Surn' in p.tag and p.score >= prob_thresh):
                    result += '{:<12}({:>12})score:{:0.3}'.format(word, p.normal_form, p.score)
                    result += ' \ '         
    if result != '':
        return result
    else:
        return 0

In [ ]:
def name_recognize_natasha(text):
    global morph
    extractor = NamesExtractor(morph)
    matches = extractor(text)
    if matches != None:
        result = ''
        for index, match in enumerate(matches):
            if match != None:
                result += str(match.fact)
    if result !='':
        return result
    else:
        return 0

In [ ]:
def name_recognize_stanza(text):
    result = ''
    global stanza_nlp
    doc = stanza_nlp(text)
    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == 'PER':
                result +=  f'entity: {ent.text}\ttype: {ent.type}' + " "
    if result != '':
        return result
    else: return 0

In [ ]:
def name_recognize_pullenti(text):
    result = ''
    processor = Processor([PERSON])
    ner_result = processor(text)
    if ner_result.matches != []:
        for match in ner_result.matches:
            result += str(match)+ ' '
    if result != '':
        return result
    else: return 0

In [26]:
s="Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). В записочках, разосланных утром с красным лакеем, было написано без различия во всех:  — отвечал, нисколько не смутясь такою встречей, вошедший князь, в придворном, шитом мундире, в чулках, башмаках и звездах, с светлым выражением плоского лица.Он говорил на том изысканном французском языке, на котором не только говорили, но и думали наши деды, и с теми, тихими, покровительственными интонациями, которые свойственны состаревшемуся в свете и при дворе значительному человеку. Он подошел к Анне Павловне, поцеловал ее руку, подставив ей свою надушенную и сияющую лысину, и покойно уселся на диване."

In [ ]:
print(name_recognize_nltk(s))

Анна        (        анна)score:1.0 \ Марии       (       мария)score:0.625 \ Василия     (     василий)score:0.923 \ 


In [ ]:
print(name_recognize_natasha(s))

Name(first=None, last='в', middle=None)Name(first=None, last='июле', middle=None)Name(first=None, last='известная', middle=None)Name(first='Анна', last='Шерер', middle='Павловна')Name(first=None, last='и', middle=None)Name(first='Марии', last='Феодоровны', middle=None)Name(first=None, last='и', middle=None)Name(first=None, last='князя', middle=None)Name(first='Василия', last=None, middle=None)Name(first=None, last='первого', middle=None)Name(first=None, last='вечер', middle=None)


In [ ]:
print(name_recognize_stanza(s))

entity: Анна Павловна Шерер	type: PER entity: Марии Феодоровны	type: PER entity: Василия	type: PER 


In [ ]:
print(name_recognize_pullenti(s))

Match(referent=PersonReferent(label='PERSON', slots=[Slot(key='SEX', value='FEMALE'), Slot(key='LASTNAME', value='ШЕРЕР'), Slot(key='FIRSTNAME', value='АННА'), Slot(key='MIDDLENAME', value='ПАВЛОВНА'), Slot(key='ATTRIBUTE', value=PersonPropertyReferent(label='PERSONPROPERTY', slots=[Slot(key='NAME', value='фрейлина')]))]), span=Span(start=40, stop=69), children=[Match(referent=PersonPropertyReferent(label='PERSONPROPERTY', slots=[Slot(key='NAME', value='фрейлина')]), span=Span(start=61, stop=69), children=[])]) Match(referent=PersonReferent(label='PERSON', slots=[Slot(key='SEX', value='FEMALE'), Slot(key='FIRSTNAME', value='МАРЬЯ'), Slot(key='FIRSTNAME', value='МАРИЯ'), Slot(key='MIDDLENAME', value='ФЕОДОРОВНА'), Slot(key='ATTRIBUTE', value=PersonPropertyReferent(label='PERSONPROPERTY', slots=[Slot(key='NAME', value='императрица')]))]), span=Span(start=85, stop=113), children=[Match(referent=PersonPropertyReferent(label='PERSONPROPERTY', slots=[Slot(key='NAME', value='императрица')]), 

# Новый раздел

In [ ]:
import math
import pandas as pd
import re

In [ ]:
def idf_tf(text):
  global df
  words={}
  sent={}
  s=0
  sum=0
  test = re.sub(r'[^А-Яа-яё0-9 ]', "", text)
  for i in test.split(" "):
    sum+=1
    if i in words.keys():
      words[i]+=1
    else:
      words[i]=1
      sent[i]=0
      text = re.sub(r'[^А-Яа-яё0-9. ]', " ", text)
      text=" "+text
      for j in text.split("."):
         j=" "+j+" "
         st=" "+i+" "
         if st in j:
            sent[i]+=1
  #print(words) 
  #print(sent) 
  for j in words.keys():
     a=(words[j]/sum)*math.log(len(sent)/sent[i])
     print(j,a)

In [ ]:
idf_tf(" Так говорила в июле 1805 года известная. Анна. Павловна Шерер, фрейлина. и приближенная Анна Анна года известная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер.")

 0.10299490206263529
Так 0.10299490206263529
говорила 0.10299490206263529
в 0.10299490206263529
июле 0.10299490206263529
1805 0.10299490206263529
года 0.20598980412527057
известная 0.20598980412527057
Анна 0.30898470618790586
Павловна 0.10299490206263529
Шерер 0.10299490206263529
фрейлина 0.10299490206263529
и 0.20598980412527057
приближенная 0.10299490206263529
императрицы 0.10299490206263529
Марии 0.10299490206263529
Феодоровны 0.10299490206263529
встречая 0.10299490206263529
важного 0.10299490206263529
чиновного 0.10299490206263529
князя 0.10299490206263529
Василия 0.10299490206263529
первого 0.10299490206263529
приехавшего 0.10299490206263529
на 0.10299490206263529
ее 0.10299490206263529
вечер 0.10299490206263529


In [ ]:
def ngl(n, text):
   for i in range(0,len(text)-n+1):
      print(text[i:i+n])

In [ ]:
ngl(3, "abcdefghj")

abc
bcd
cde
def
efg
fgh
ghj


In [ ]:
def ngw(n, text):
   a=text.split() 
   for i in range(0, len(a)-n+1):
      b=""
      for j in range (i, i+n):
          b=b+" "+a[j]
      print(b)

In [ ]:
ngw(3, "01 02 003 4 50 68 70 88 9090")

 01 02 003
 02 003 4
 003 4 50
 4 50 68
 50 68 70
 68 70 88
 70 88 9090
